# TODO

you can't actually test this model therefore create a version with just df

In [1]:
import os
import sklearn
import seaborn as sns
import numpy as np
import pandas as pd
import re
import pickle
import json
import math
import random
from tqdm import tqdm
import datetime
import time

from PyAstronomy import pyasl

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, MissingIndicator

from sklearn.preprocessing import (
    StandardScaler, 
    MinMaxScaler, 
    MaxAbsScaler, 
    # RobustScalar,
    Normalizer,
    QuantileTransformer,
    PowerTransformer,
    OneHotEncoder, 
    OrdinalEncoder,
    LabelEncoder
)

from sklearn.utils import all_estimators

from sklearn.base import (
    RegressorMixin, 
    ClassifierMixin,
    TransformerMixin
)

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    auc,
    roc_auc_score,
    f1_score,
    r2_score,
    mean_squared_error,
    classification_report
)

import warnings
import xgboost
import catboost
import lightgbm

import tensorflow as tf

warnings.filterwarnings("ignore")
pd.set_option("display.precision", 2)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

removed_classifiers = [
    "ClassifierChain",
    "ComplementNB",
    # "GradientBoostingClassifier",
    "GaussianProcessClassifier",
    "HistGradientBoostingClassifier",
    # "MLPClassifier",
    "LogisticRegressionCV", 
    "MultiOutputClassifier", 
    "MultinomialNB", 
    "OneVsOneClassifier",
    "OneVsRestClassifier",
    "OutputCodeClassifier",
    "RadiusNeighborsClassifier",
    "VotingClassifier",
    "CategoricalNB",
    "StackingClassifier",
    "NuSVC",
]

removed_regressors = [
    "TheilSenRegressor",
    "ARDRegression", 
    "CCA", 
    "IsotonicRegression", 
    "StackingRegressor",
    "MultiOutputRegressor", 
    "MultiTaskElasticNet", 
    "MultiTaskElasticNetCV", 
    "MultiTaskLasso", 
    "MultiTaskLassoCV", 
    "PLSCanonical", 
    "PLSRegression", 
    "RadiusNeighborsRegressor", 
    "RegressorChain", 
    "VotingRegressor", 
]

CLASSIFIERS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], ClassifierMixin) and (est[0] not in removed_classifiers))
]


REGRESSORS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], RegressorMixin) and (est[0] not in removed_regressors))
]

REGRESSORS.append(("XGBRegressor", xgboost.XGBRegressor))
REGRESSORS.append(("LGBMRegressor", lightgbm.LGBMRegressor))
REGRESSORS.append(('CatBoostRegressor', catboost.CatBoostRegressor))

CLASSIFIERS.append(("XGBClassifier", xgboost.XGBClassifier))
CLASSIFIERS.append(("LGBMClassifier", lightgbm.LGBMClassifier))
CLASSIFIERS.append(('CatBoostClassifier', catboost.CatBoostClassifier))

TRANSFOMER_METHODS = [
    ("StandardScaler", StandardScaler), 
    ("MinMaxScaler", MinMaxScaler), 
    ("MaxAbsScaler", MaxAbsScaler), 
    # ("RobustScalar", RobustScalar),
    ("Normalizer", Normalizer),
    ("QuantileTransformer", QuantileTransformer),
    ("PowerTransformer", PowerTransformer),
]

def adjusted_rsquared(r2, n, p):
    return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

2023-05-19 08:58:40.115286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 08:58:40.244479: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-19 08:58:40.681628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-19 08:58:40.681718: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

References:

1. https://discuss.huggingface.co/t/what-does-increasing-number-of-heads-do-in-the-multi-head-attention/1847/2

In [2]:
df_train = pd.read_excel("dataset_rnn.xlsx", sheet_name="train")
df_test = pd.read_excel("dataset_rnn.xlsx", sheet_name="test")
display(df_train)
display(df_test)

,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max,rnn_encoding
0,Al92Nd6Ni2,10.61,0.32,0.20,-0.17,0.00,0.00
1,Zr26Ti10Cu8Ni8Be20Y4Mg24,13.89,1.78,0.17,-0.12,5.00,5.11
2,Gd55Ni22Mn3Al20,11.82,1.09,0.26,-0.18,2.00,2.24
3,Ag53.8Mg15.4Ca30.8,10.05,0.98,0.25,-0.28,0.80,0.86
4,Ni60Nb20Zr20,19.88,0.95,0.17,-0.13,0.50,0.47
...,...,...,...,...,...,...,...
1054,Mg65Cu7.5Ni7.5Ag5Zn5Gd7.5Y2.5,9.89,1.25,0.18,-0.16,13.00,13.12
1055,Mg65Cu7.5Ni7.5Ag5Zn5Gd2.5Y7.5,9.96,1.25,0.17,-0.16,9.50,9.80
1056,Mg80Ni10Nd10,9.39,0.64,0.25,-0.09,0.60,0.28
1057,Cu45.5Mg31.8Ca22.7,10.66,1.06,0.34,-0.25,1.25,1.16


,bmg_alloy,delta_h_mix,delta_s_mix,delta_d,delta_e,actual_d_max,predicted_d_max,rnn_encoding
0,Co14Ni69P17,14.24,0.83,0.12,0.04,0.00,0.02,0.00
1,C14Fe66W20,30.81,0.87,0.14,-0.13,0.00,0.00,0.00
2,Cu15La30Mg55,8.64,0.97,0.31,-0.13,0.00,1.77,0.00
3,Al87.5Ca2.5Ni10,11.30,0.44,0.21,-0.24,0.00,-0.01,0.00
4,B36C8Co56,35.47,0.89,0.11,-0.14,0.00,-0.03,0.00
...,...,...,...,...,...,...,...,...
257,Gd55Ni25Al20,11.94,1.00,0.27,-0.19,2.00,1.36,3.91
258,Mg65Ni20La15,10.04,0.89,0.29,-0.14,0.50,0.44,2.97
259,B8Fe89Ni3,16.80,0.41,0.16,-0.02,0.00,0.02,0.00
260,La62Al14Cu18Ag6,8.44,1.05,0.36,-0.23,5.00,5.77,5.19


## 1. Test the capability of our d_max predictor

In [3]:
# constants

n_elements = 118
alloy_max_len = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

max_factor = max(max(df_train["actual_d_max"]), max(df_test["actual_d_max"]))

print(max_factor)

with open("parameters.json") as json_file:
    parameters = json.load(json_file)

30.0


In [4]:
# basic utility functions to calculate features

def get_elements_and_compositions(x, verbose = -1):
    # seperating atoms from composition
    s = re.sub(r'[^\w\s]','',x)
    s = re.sub('\d',' ',s)
    elements = np.array([i for i in s.split(' ') if i != ""])
    if verbose > 0:
        print('\nElements in BMG are : ', elements)

    compositions = re.findall(r"[-+]?\d*\.\d+|\d+", x)
    compositions = [float(i) for i in compositions]
    if verbose > 0:
        print('Compositions: ', compositions)
    
    return elements, compositions

def diff(alloy):
    # making ranges for each atom
    ranges = {}
    for i in alloy:
        ranges[i] = 0.88 * parameters[i]["ar"]
    # compiling scoring matrix
    score = {}
    for i in alloy:
        current_score = {}
        for j in alloy:
            if parameters[i]["ar"] < ranges[j]:
                current_score[j] = -1
            elif parameters[i]["ar"] > parameters[j]["ar"]:
                current_score[j] = 1
            else:
                current_score[j] = 0
        score[i] = current_score

    big = []
    small = []
    # separating into big and small based on scoring matrix
    for i in score:
        total_sum = 0
        for j in score[i]:
            total_sum = total_sum + score[i][j]
        if total_sum > 0:
            big.append(i)
        else:
            small.append(i)

    if len(big) == 0 or len(small) == 0:
        print(score)
    return big, small
    
# finds the paramater deltaE
def electro(elements_and_compositions, alloy):
    summation_of_product_of_composition_and_electronegativity = 0 # summation of product of composition and electro negativity
    summation_of_composition = 0 # summation of compositions
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_composition = summation_of_composition + elements_and_compositions[i]
            
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_product_of_composition_and_electronegativity = summation_of_product_of_composition_and_electronegativity + elements_and_compositions[i] * parameters[i]['en']
    return summation_of_product_of_composition_and_electronegativity / summation_of_composition

# finds the paramater deltaD
def comps(elements_and_compositions, alloy):
    summation_of_product_of_composition_and_atomicradii = 0 # summation of product of composition and atomic radii
    summation_of_compositions = 0 # summation of compositions
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_compositions = summation_of_compositions + elements_and_compositions[i]
    for i in elements_and_compositions:
        if i in alloy:
            summation_of_product_of_composition_and_atomicradii = summation_of_product_of_composition_and_atomicradii + elements_and_compositions[i] * parameters[i]['ar']
    return summation_of_product_of_composition_and_atomicradii / summation_of_compositions

def prepare_params(alloy):
    # seperating atoms from composition
    s = re.sub(r'[^\w\s]','', alloy)
    s = re.sub('\d', ' ', s)
    elements = np.array([i for i in s.split(' ') if i in parameters]) # elements list
    # print('\nElements in BMG are : ', elements)

    compositions = re.findall(r"[-+]?\d*\.\d+|\d+", alloy)
    compositions = [float(i) for i in compositions]
    # print('Compositions: ', compositions)

    elements_and_compositions = dict(zip(elements, compositions))
    s_mix = 0
    h_mix = 0

    for i in elements_and_compositions:
        s_mix = s_mix + (elements_and_compositions[i] / 100) * (math.log((elements_and_compositions[i] / 100)))
        h_mix = h_mix + (elements_and_compositions[i] / 100) * parameters[i]['enthalphy']
    s_mix = -1*s_mix

    big, small = diff(elements)
    # print("big atoms : ", big)
    # print("small atoms : ", small)
    delta_d = (comps(elements_and_compositions,big) - comps(elements_and_compositions,small)) / (comps(elements_and_compositions,big))
    delta_e = (electro(elements_and_compositions,big) - electro(elements_and_compositions,small)) / (electro(elements_and_compositions,big) + electro(elements_and_compositions,small))
    return h_mix, s_mix, delta_d, delta_e, elements[np.argmax(compositions)]

In [5]:
# utility functions for encoding

# find element index from all_elements, e.g. "He" = 2
def element_to_index(element):
    try:
        atomic_number = pyasl.AtomicNo()
        return atomic_number.getAtomicNo(element)
    except:
        return "END"
    
def index_to_element(index):
    try:
        atomic_number = pyasl.AtomicNo()
        return atomic_number.getElSymbol(index)
    except:
        return "END"
    
def alloy_to_1d_tensor(alloy_str, alloy_max_len = alloy_max_len):
    tensor = torch.zeros(alloy_max_len)
    elements, compositions = get_elements_and_compositions(alloy_str)
    i = 0
    for idx in range(0, len(elements) + len(compositions), 2):
        tensor[idx] = element_to_index(elements[i])
        tensor[idx + 1] = compositions[i]
        i += 1
    return tensor

def elements_to_1d_tensor(elements, element_max_len = alloy_max_len // 2):
    tensor = torch.zeros(element_max_len)
    for idx in range(len(elements)):
        tensor[idx] = element_to_index(elements[idx])
    # print(tensor)
    return tensor

def composition_to_1d_tensor(compositions, composition_max_len = alloy_max_len // 2):
    tensor = torch.zeros(composition_max_len)
    for idx in range(len(compositions)):
        tensor[idx] = compositions[idx]
    # print(tensor)
    return tensor

def combine_elements_and_composition_to_alloy(elements, compositions):
    alloy = ""
    for idx in range(len(elements)):
        alloy += str(index_to_element(elements[idx])) + str(compositions[idx])[:4]
    return alloy.split("END")[0]

In [6]:
class SequenceToSigmoid(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

def predict_d_max(alloy):
    input_size = 20
    hidden_size = 128
    learning_rate = 0.001
    num_epochs = 2000
    batch_size = 32
    num_layers = 3
    
    loaded_rnn_encoder = SequenceToSigmoid(input_size, hidden_size, num_layers)
    loaded_rnn_encoder.load_state_dict(torch.load("rnn_fully_trained.pt"))
    loaded_rnn_encoder.eval()
    
    loaded_X_transformer = pickle.load(open("catboost_X_quantiletransformer.pkl", "rb"))
    loaded_y_transformer = pickle.load(open("catboost_y_quantiletransformer.pkl", "rb"))
    loaded_catboost_predictor = pickle.load(open("catboost_model.pkl", "rb"))
    
    alloy_tensor = alloy_to_1d_tensor(alloy)
    rnn_prediction = loaded_rnn_encoder(torch.atleast_2d(alloy_tensor)).squeeze().item()
    rnn_prediction *= max_factor
    # print(rnn_prediction)
    
    h_mix, s_mix, delta_d, delta_e, max_element = prepare_params(alloy)
    # print(h_mix, s_mix, delta_d, delta_e, max_element)
    
    transformed_input = loaded_X_transformer.transform(np.array([h_mix, s_mix, delta_d, delta_e, rnn_prediction]).reshape(1, -1))
    # print(transformed_input)
    transformed_output = loaded_catboost_predictor.predict(transformed_input)
    # print(transformed_output)
    output = loaded_y_transformer.inverse_transform(transformed_output.reshape(1, -1)).item()
    return output

In [7]:
predict_d_max("Zr26Ti10Cu8Ni8Be20Y4Mg24")

5.0

## 2. Generate optimized BMGs

In [8]:
X_train = []
y_train = []

X_test = []
y_test = []

for i in range(df_train.shape[0]):
    elements, compositions = get_elements_and_compositions(df_train.loc[i, "bmg_alloy"])
    X_train.append(elements_to_1d_tensor(elements))
    y_train.append(composition_to_1d_tensor(compositions))

X_train = torch.stack(X_train)
y_train = torch.stack(y_train)

print(X_train.shape, X_train.dtype, y_train.shape, y_train.dtype)

torch.Size([1059, 10]) torch.float32 torch.Size([1059, 10]) torch.float32


In [9]:
for i in range(df_test.shape[0]):
    elements, compositions = get_elements_and_compositions(df_test.loc[i, "bmg_alloy"])
    X_test.append(elements_to_1d_tensor(elements))
    y_test.append(composition_to_1d_tensor(compositions))

X_test = torch.stack(X_test)
y_test = torch.stack(y_test)

print(X_test.shape, X_test.dtype, y_test.shape, y_test.dtype)

torch.Size([262, 10]) torch.float32 torch.Size([262, 10]) torch.float32


In [10]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Seq2Seq, self).__init__()

        self.hidden_size = hidden_size

        # Encoder
        self.encoder = nn.LSTM(input_size, hidden_size)

        # Decoder
        self.decoder = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, target_seq):
        
        input_seq = input_seq.unsqueeze(1)
        target_seq = target_seq.unsqueeze(1)
        
        # Encode the input sequence
        _, hidden = self.encoder(input_seq)

        # Decode the output sequence
        output, _ = self.decoder(target_seq, hidden)
        output = self.fc(output)
        
        output = F.softmax(output, dim=2)
        output = output * (input_seq != 0).float()

        # Normalize the output sequence to sum up to 1
        output_sum = output.sum(dim=2, keepdim=True)
        output = output / output_sum
        
        return output.squeeze(1)

def custom_loss(output, target, predicted_d_max):
    # Calculate the loss based on the output and target
    loss = F.mse_loss(output, target)
    
    # print("loss before", loss)
    # Scale the loss based on the predicted_d_max
    normalized_d_max = predicted_d_max / max_factor
    scaled_loss = loss / (normalized_d_max + 0.1)
    
    return Variable(torch.tensor(normalized_d_max * -1), requires_grad = True)
    # return scaled_loss

In [11]:
input_size = alloy_max_len // 2
hidden_size = 32
output_size = alloy_max_len // 2

model = Seq2Seq(input_size, hidden_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def random_training_example(input_data, output_data):
    random_number = random.randint(0, input_data.shape[0] - 1)
    elements_tensor, compositions_tensor = input_data[random_number].view(1, -1), output_data[random_number].view(1, -1)
    
    # print(elements_tensor, compositions_tensor)
    element_input = elements_tensor[0].tolist()
    composition_output = compositions_tensor[0].tolist()
    
    elements = []
    for idx in range(len(element_input)):
        elements.append(index_to_element(element_input[idx]))
    alloy_str = ""
    
    for idx in range(min(len(composition_output), len(elements))):
        alloy_str += str(elements[idx]) + str(composition_output[idx])
    
    alloy_str = alloy_str.split("END")[0]
    print("Actual alloy:", alloy_str)
    print("Actual d_max:", df_train.loc[random_number, "actual_d_max"])
    
    return elements_tensor, compositions_tensor, df_train.loc[random_number, "actual_d_max"]

random_training_example(X_train, y_train)

Actual alloy: Cu60.0Zr30.0Ti10.0
Actual d_max: 4.0


(tensor([[29., 40., 22.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 tensor([[60., 30., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
 4.0)

In [12]:
num_epochs = 2000
all_losses = []
current_loss = 0
plot_every = 100
print_every = 1000

wins_and_losses = []
better_generations = []

for epoch in range(num_epochs):
    try:
        model.train()
        optimizer.zero_grad()
        rand_input_seq, rand_output_seq, actual_d_max = random_training_example(X_train, y_train)
        output_seq = model(rand_input_seq, rand_input_seq)
        # print(np.sum(np.array(output_seq[0].tolist()) * 100))
        generated_alloy = combine_elements_and_composition_to_alloy(rand_input_seq[0].tolist(), list(np.array(output_seq[0].tolist()) * 100))
        # print("Generated Alloy", generated_alloy)
        generated_alloy_d_max = predict_d_max(generated_alloy)
        # print("Generated Alloy d_max", generated_alloy_d_max)
        # print("LOSS", loss)
        
        if generated_alloy_d_max > actual_d_max:
            wins_and_losses.append("win")
            better_generations.append(generated_alloy)
        else:
            wins_and_losses.append("loss")
        
        loss = custom_loss(output_seq, rand_output_seq, generated_alloy_d_max)
        # loss = Variable(0.0, requires_grad = True) if generated_alloy_d_max > actual_d_max else Variable(1.0, requires_grad = True)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if (epoch + 1) % print_every == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        if (epoch + 1) % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0
    except Exception as e:
        print(e)

Actual alloy: B9.0Fe83.0Si8.0
Actual d_max: 0.0
Actual alloy: Co64.80000305175781Fe7.199999809265137B19.200000762939453Si4.800000190734863Nb4.0
Actual d_max: 2.0
Actual alloy: Ni30.0Si30.0Ti40.0
Actual d_max: 0.0
Actual alloy: Fe58.79999923706055Cr9.800000190734863Mo8.819999694824219C12.739999771118164B5.880000114440918Er1.9600000381469727Pb2.0
Actual d_max: 5.0
Actual alloy: Al85.0Fe3.0Ni12.0
Actual d_max: 0.0
Actual alloy: La66.0Al14.0Cu10.0Ni10.0
Actual d_max: 1.5
Actual alloy: Mg59.5Cu22.899999618530273Ag6.599999904632568Gd11.0
Actual d_max: 27.0
Actual alloy: Fe76.0Si9.0B10.0P5.0
Actual d_max: 2.5
Actual alloy: Ca65.0Mg5.0Cu30.0
Actual d_max: 0.5
Actual alloy: Be2.5Zr97.5
Actual d_max: 0.0
Actual alloy: Al40.0Cu30.0La30.0
Actual d_max: 0.0
Actual alloy: Ca50.0Mg30.0Cu20.0
Actual d_max: 2.0
Actual alloy: Al95.0Ce2.0Co3.0
Actual d_max: 0.0
Actual alloy: Ti50.0Ni15.0Cu25.0Sn3.0Be7.0
Actual d_max: 2.0
Actual alloy: Ti43.150001525878906Zr9.59000015258789Cu36.2400016784668Ni9.0600004196

Actual alloy: Ag53.79999923706055Mg15.399999618530273Ca30.799999237060547
Actual d_max: 0.8
Actual alloy: Zr63.5Al10.699999809265137Cu10.699999809265137Ni15.100000381469727
Actual d_max: 6.0
Actual alloy: B8.0Cu3.0Fe89.0
Actual d_max: 0.0
Actual alloy: C5.0Co70.0Si25.0
Actual d_max: 0.0
Actual alloy: Al65.0Cu20.0V15.0
Actual d_max: 0.0
Actual alloy: Al2.0Ce6.0Cu92.0
Actual d_max: 0.0
Actual alloy: Al82.0Mn4.0Ni14.0
Actual d_max: 0.0
Actual alloy: Mg65.0Cu15.0Ag10.0Gd10.0
Actual d_max: 7.5
Actual alloy: Al15.0Fe56.70000076293945Tb28.299999237060547
Actual d_max: 0.0
Actual alloy: Cu47.0Ti33.0Zr9.0Nb2.0Ni8.0Si1.0
Actual d_max: 5.0
Actual alloy: Ni32.0P20.0Pt48.0
Actual d_max: 0.0
Actual alloy: B2.0Fe78.0Ni20.0
Actual d_max: 0.0
Actual alloy: Pd79.0Cu4.0Ag2.0Si10.0P5.0
Actual d_max: 5.0
Actual alloy: Zr41.0Ti14.0Cu12.5Ni8.0Be22.5C2.0
Actual d_max: 5.0
Actual alloy: Fe50.400001525878906Co21.600000381469727B19.200000762939453Si4.800000190734863Nb4.0
Actual d_max: 3.5
Actual alloy: Be25.0Si1

Actual alloy: Al45.0Cr20.0Si35.0
Actual d_max: 0.0
Actual alloy: Al33.599998474121094Ca66.4000015258789
Actual d_max: 1.0
Actual alloy: Cu70.0Ni10.0Zr20.0
Actual d_max: 0.0
Actual alloy: Al80.0Ce10.0Mn10.0
Actual d_max: 0.0
Actual alloy: La10.0Mg85.0Ni5.0
Actual d_max: 0.0
Actual alloy: Al82.0Fe4.0Ni14.0
Actual d_max: 0.0
Actual alloy: Cu47.0Ti33.0Zr9.0Nb2.0Ni8.0Si1.0
Actual d_max: 5.0
Actual alloy: Mg58.5Cu30.5Y11.0
Actual d_max: 9.0
Actual alloy: Cu10.0Ni10.0Ti80.0
Actual d_max: 0.0
Actual alloy: Pd4.0Zr48.0Cu32.0Al8.0Ag8.0
Actual d_max: 25.0
Actual alloy: Al65.0Ce15.0Fe20.0
Actual d_max: 0.0
Actual alloy: B50.0C4.0Ni46.0
Actual d_max: 0.0
Actual alloy: La62.0Al14.0Cu10.0Ag2.0Ni6.0Co6.0
Actual d_max: 16.0
Actual alloy: Co12.5Ni68.5P19.0
Actual d_max: 0.0
Actual alloy: Ni32.0P20.0Pt48.0
Actual d_max: 0.0
Actual alloy: Fe76.0Si9.0B10.0P5.0
Actual d_max: 2.5
Actual alloy: B2.0Fe80.0Mo18.0
Actual d_max: 0.0
Actual alloy: C14.0Co40.0Cr46.0
Actual d_max: 0.0
Actual alloy: Al88.0Co6.0Fe6.0


Actual alloy: Zr49.0Ti17.0Ni20.0Cu14.0
Actual d_max: 3.0
Actual alloy: Pd73.5Cu10.0Si16.5
Actual d_max: 2.0
Actual alloy: Fe72.0W2.0Y4.0B22.0
Actual d_max: 3.0
Actual alloy: Ag45.0Ce2.0Cu53.0
Actual d_max: 0.0
Actual alloy: Al60.0Ce20.0Fe20.0
Actual d_max: 0.0
Actual alloy: Al2.0Cu90.0Nd8.0
Actual d_max: 0.0
Actual alloy: Ni59.0Zr19.0Ti11.0Si2.0Sn3.0Nb6.0
Actual d_max: 3.0
Actual alloy: Al90.0Ca5.0Ni5.0
Actual d_max: 0.0
Actual alloy: Zr50.0Cu40.0Ag10.0
Actual d_max: 5.0
Actual alloy: B12.5Co80.5Si7.0
Actual d_max: 0.0
Actual alloy: La62.5Al12.5Ag5.0Cu17.5Fe2.5
Actual d_max: 7.0
Actual alloy: Al77.0Ce18.0Fe5.0
Actual d_max: 0.0
Actual alloy: Fe76.0Si9.0B10.0P5.0
Actual d_max: 2.5
Actual alloy: Fe70.55999755859375Nb3.9200000762939453B19.600000381469727Si3.9200000762939453Y2.0
Actual d_max: 2.0
Actual alloy: Ce60.0Al10.0Cu20.0Ni10.0
Actual d_max: 1.0
Actual alloy: Fe71.0Mo3.0Nb2.0P12.0C10.0B2.0
Actual d_max: 1.0
Actual alloy: Ni6.0Zr48.0Cu30.0Al8.0Ag8.0
Actual d_max: 25.0
Actual alloy: A

Actual alloy: Cu5.0Ni5.0Ti90.0
Actual d_max: 0.0
Actual alloy: Ti34.0Zr11.0Cu47.0Ni8.0
Actual d_max: 4.5
Actual alloy: Fe15.0Pd68.5Si16.5
Actual d_max: 0.0
Actual alloy: C18.0Co68.0W14.0
Actual d_max: 0.0
Actual alloy: Cu47.0Ti33.0Zr11.0Ni8.0Si1.0
Actual d_max: 4.0
Actual alloy: B17.0C16.0Ni67.0
Actual d_max: 0.0
Actual alloy: Cu5.0Ni5.0Ti90.0
Actual d_max: 0.0
Actual alloy: Fe72.0C7.0Si3.299999952316284B5.0P8.699999809265137Ga4.0
Actual d_max: 2.0
Actual alloy: Er55.0Al25.0Co20.0
Actual d_max: 5.0
Actual alloy: La62.0Al14.0Cu13.34000015258789Ag2.6600000858306885Ni4.0Co4.0
Actual d_max: 16.0
Actual alloy: La62.0Al14.0Cu22.0Ag2.0
Actual d_max: 5.0
Actual alloy: Fe58.0Cr5.0Mo14.0Er2.0C15.0B6.0
Actual d_max: 6.0
Actual alloy: Al10.0Mg60.0Zn30.0
Actual d_max: 0.0
Actual alloy: C18.0Mo28.0Ni54.0
Actual d_max: 0.0
Actual alloy: Mg57.0Ni26.0La17.0
Actual d_max: 2.0
Actual alloy: Pd81.5Cu2.0Si16.5
Actual d_max: 2.0
Actual alloy: Ca50.0Mg15.0Zn10.0Cu25.0
Actual d_max: 10.0
Actual alloy: Cu54.0Z

...

Actual alloy: Al82.0Sm18.0
Actual d_max: 0.0
Actual alloy: Ni59.0Zr14.0Ti13.0Si2.0Sn3.0Nb9.0
Actual d_max: 2.0
Actual alloy: Fe87.0Zr13.0
Actual d_max: 0.0
Actual alloy: Al75.0Ce5.0Co20.0
Actual d_max: 0.0
Actual alloy: Co9.0Ni74.0P17.0
Actual d_max: 0.0
Actual alloy: Cu60.0Hf17.5Ti22.5
Actual d_max: 3.0
Actual alloy: Cu1.0Ni75.0P24.0
Actual d_max: 0.0
Actual alloy: Al91.0Fe6.0Ni3.0
Actual d_max: 0.0
Actual alloy: Al75.0Ce10.0Co15.0
Actual d_max: 0.0
Actual alloy: Al60.0Ce20.0Ni20.0
Actual d_max: 0.0
Actual alloy: Ni40.0Si30.0Ti30.0
Actual d_max: 0.0
Actual alloy: Fe71.0Nb4.0Hf3.0Y2.0B20.0
Actual d_max: 4.0
Actual alloy: Nb30.0Si20.0Ti50.0
Actual d_max: 0.0
Actual alloy: Be30.0Si10.0Ti60.0
Actual d_max: 0.0
Actual alloy: Mg87.5Y12.5
Actual d_max: 0.0
Actual alloy: Fe69.83999633789062Nb3.880000114440918B19.399999618530273Si3.880000114440918Y3.0
Actual d_max: 4.0
Actual alloy: Al94.0Ce4.0Co2.0
Actual d_max: 0.0
Actual alloy: Au49.0Ag5.5Pd2.299999952316284Cu26.899999618530273Si16.29999923

Actual alloy: Mg70.0Ni20.0La10.0
Actual d_max: 0.4
Actual alloy: Ti47.5Zr2.5Cu42.5Ni7.5
Actual d_max: 1.5
Actual alloy: Al85.0Fe8.0Si7.0
Actual d_max: 0.0
Actual alloy: Mg71.0Ni15.0Gd10.0Ag4.0
Actual d_max: 4.0
Actual alloy: Be35.0Ti65.0
Actual d_max: 0.0
Actual alloy: Ni50.0Ti50.0
Actual d_max: 0.0
Actual alloy: Y4.0Fe72.0B22.0Ta2.0
Actual d_max: 4.0
Actual alloy: La62.5Al12.5Ag5.0Cu17.5Co2.5
Actual d_max: 8.0
Actual alloy: Ce15.0Cu10.0Mg75.0
Actual d_max: 0.0
Actual alloy: La55.0Al25.0Co20.0
Actual d_max: 5.0
Actual alloy: Nd50.0Ni50.0
Actual d_max: 0.0
Actual alloy: Al20.0Cu10.0Ti70.0
Actual d_max: 0.0
Actual alloy: C10.0Fe74.0W16.0
Actual d_max: 0.0
Actual alloy: La5.0Mg90.0Ni5.0
Actual d_max: 0.0
Actual alloy: Mg58.5Cu30.5Gd11.0
Actual d_max: 8.0
Actual alloy: Ag38.400001525878906Mg30.799999237060547Ca30.799999237060547
Actual d_max: 0.5
Actual alloy: Al30.0Cu10.0Ti60.0
Actual d_max: 0.0
Actual alloy: Al4.0B10.0Co86.0
Actual d_max: 0.0
Actual alloy: C16.0Co52.0Cr32.0
Actual d_max:

Actual alloy: La62.0Cu24.0Al10.5Mg3.5
Actual d_max: 10.0
Actual alloy: B32.0C20.0Co48.0
Actual d_max: 0.0
Actual alloy: Al94.0Co4.0Ni2.0
Actual d_max: 0.0
Actual alloy: La60.0Al15.0Ag5.0Cu20.0
Actual d_max: 5.0
Actual alloy: Ag31.0Cu55.0Fe14.0
Actual d_max: 0.0
Actual alloy: Ag28.0Cu45.0Fe27.0
Actual d_max: 0.0
Actual alloy: Cu75.0Ti25.0
Actual d_max: 0.0
Actual alloy: Ni6.0Zr48.0Cu30.0Al8.0Ag8.0
Actual d_max: 25.0
Actual alloy: Cu5.0Mg80.0Nd15.0
Actual d_max: 0.0
Actual alloy: Cu40.0Hf10.0Ni50.0
Actual d_max: 0.0
Actual alloy: Ni59.0Zr15.0Ti13.0Si2.0Sn3.0Nb8.0
Actual d_max: 2.0
Actual alloy: La5.0Mg90.0Ni5.0
Actual d_max: 0.0
Actual alloy: Ca55.0Mg20.0Zn25.0
Actual d_max: 2.0
Actual alloy: Ni61.0Zr28.0Nb7.0Al4.0
Actual d_max: 1.0
Actual alloy: La55.0Al25.0Ag5.0Cu15.0
Actual d_max: 3.0
Actual alloy: B22.0C5.0Ni73.0
Actual d_max: 0.0
Actual alloy: Al30.0Cu10.0Ti60.0
Actual d_max: 0.0
Actual alloy: Y55.0Al25.0Co20.0
Actual d_max: 2.0
Actual alloy: Ni59.0Zr20.0Ti16.0Si5.0
Actual d_max: 2.

Actual alloy: Fe57.599998474121094Co14.399999618530273B19.200000762939453Si4.800000190734863Nb4.0
Actual d_max: 2.5
Actual alloy: Cu20.0Ni40.0Ti40.0
Actual d_max: 0.0
Actual alloy: Nd50.0Ni50.0
Actual d_max: 0.0
Actual alloy: C16.0Co64.0W20.0
Actual d_max: 0.0
Actual alloy: B16.0Cu10.0Fe74.0
Actual d_max: 0.0
Actual alloy: Al80.0Cr10.0Si10.0
Actual d_max: 0.0
Actual alloy: C20.0Fe70.0W10.0
Actual d_max: 0.0
Actual alloy: Au49.0Ag5.5Pd2.299999952316284Cu26.899999618530273Si16.299999237060547
Actual d_max: 5.0
Actual alloy: Al88.0Co3.0Ni9.0
Actual d_max: 0.0
Actual alloy: Cu1.0Mg89.0Y10.0
Actual d_max: 0.0
Actual alloy: Ca70.0Mg20.0Cu10.0
Actual d_max: 0.5
Actual alloy: Zr45.0Ti21.0Ni34.0
Actual d_max: 0.68
Actual alloy: Pd77.5Cu6.0Si16.5
Actual d_max: 1.5
Actual alloy: Ag50.0Fe50.0
Actual d_max: 0.0
Actual alloy: Al94.0Mn2.0Ni4.0
Actual d_max: 0.0
Actual alloy: Pd79.0Cu5.0Ag1.0Si10.0P5.0
Actual d_max: 4.0
Actual alloy: Ag58.0Cu32.0Fe10.0
Actual d_max: 0.0
Actual alloy: Al70.0Nd20.0Ni10.

Actual alloy: Cu42.0Zr42.0Ag16.0
Actual d_max: 2.0
Actual alloy: B8.0Fe87.0Nb5.0
Actual d_max: 0.0
Actual alloy: Al90.0Fe8.0Si2.0
Actual d_max: 0.0
Actual alloy: La66.0Al14.0Cu10.0Ni10.0
Actual d_max: 1.5
Actual alloy: Cu80.0Ti20.0
Actual d_max: 0.0
Actual alloy: Al10.0Ce2.0Cu88.0
Actual d_max: 0.0
Actual alloy: B5.0C15.0Ni80.0
Actual d_max: 0.0
Actual alloy: Cu45.0Zr35.0Ag10.0Hf10.0
Actual d_max: 4.0
Actual alloy: Fe65.5Cr4.0Mo4.0Ga4.0P12.0C5.0B5.5
Actual d_max: 3.0
Actual alloy: Zr50.0Cu43.0Ag7.0
Actual d_max: 4.0
Actual alloy: Mg57.0Cu31.0Y6.599999904632568Nd5.400000095367432
Actual d_max: 14.0
Actual alloy: B20.0C8.0Ni72.0
Actual d_max: 0.0
Actual alloy: Al80.0Ce10.0V10.0
Actual d_max: 0.0
Actual alloy: Fe76.0Si9.0B10.0P5.0
Actual d_max: 2.5
Actual alloy: Zr41.0Ti14.0Cu12.5Ni8.0Be22.5C2.0
Actual d_max: 5.0
Actual alloy: Ca58.0Mg18.0Cu24.0
Actual d_max: 6.0
Actual alloy: Ca50.0Mg30.0Cu20.0
Actual d_max: 2.0
Actual alloy: Cu46.400001525878906Ag11.600000381469727Zr35.0Ti7.0
Actual d_m

In [13]:
from collections import Counter
Counter(wins_and_losses)

Counter({'loss': 1667, 'win': 333})

In [15]:
print(better_generations)

['Co11.1Fe16.7B21.7Si19.4Nb30.9', 'La20.5Al19.9Cu37.9Ni21.4', 'Fe18.8Si20.6B33.3P27.1', 'Ca29.2Mg27.4Cu43.2', 'Ti16.6Ni19.5Cu23.0Sn18.2Be22.4', 'Ti9.85Zr15.5Cu19.6Ni26.0Sn28.9', 'Ni17.0Zr17.7Ti18.5Si12.5Sn15.8Nb18.2', 'Ni10.3Zr15.7Ti20.7Si22.3Sn30.8', 'Cu41.7La58.2', 'Ni17.0Zr17.7Ti18.5Si12.5Sn15.8Nb18.2', 'Fe10.9Tb18.0B21.9Si20.2Nb28.7', 'Al27.5Mg26.3Zn46.1', 'Ni17.0Zr17.7Ti18.5Si12.5Sn15.8Nb18.2', 'Fe14.7W36.3Y23.3B25.5', 'Fe19.0Mo33.6B25.6Dy21.7', 'Ni11.2Zr20.3Ti15.7Nb16.6Al35.9', 'Fe16.0Mo17.1Nb15.8P11.1C23.4B16.3', 'Co11.1Fe16.7B21.7Si19.4Nb30.9', 'Cu42.2Zr57.7', 'Y18.1Fe20.8B32.7Ti28.2', 'Al27.5Mg26.3Zn46.1', 'Fe19.0Mo33.6B25.6Dy21.7', 'Fe10.9Tb18.0B21.9Si20.2Nb28.7', 'Fe17.8B22.1Si38.0Nb21.8', 'Cu24.6Zr22.3Ag31.6Hf21.3', 'Ni11.2Zr20.3Ti15.7Nb16.6Al35.9', 'Fe17.8Cr14.6Mo21.6P11.6C21.5B12.6', 'Ti17.9Ni16.8Cu24.7Sn14.3Be13.7Zr12.3', 'Ni17.0Zr17.7Ti18.5Si12.5Sn15.8Nb18.2', 'Fe14.7W36.3Y23.3B25.5', 'Fe10.9Co16.8B21.7Si19.8Nb30.6', 'Ca29.2Mg27.4Cu43.2', 'Ag19.5Mg19.6Ca38.9Cu21.8', 'Au